In [1]:
#transfer learning

In [1]:
# -------------------------
# 1. IMPORT REQUIRED LIBRARIES
# -------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam


In [4]:

# -------------------------
# 2. SET PATHS
# -------------------------
train_dir = r"C:\Users\saira\Desktop\DL-Practical\drive-download-20251107T051626Z-1-001\Object Detection(Ass6)\caltech-101-img\train"
test_dir = r"C:\Users\saira\Desktop\DL-Practical\drive-download-20251107T051626Z-1-001\Object Detection(Ass6)\caltech-101-img\test"
weights_path = r"C:\Users\saira\Desktop\DL-Practical\drive-download-20251107T051626Z-1-001\Object Detection(Ass6)\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [5]:
# -------------------------
# 3. LOAD AND PREPROCESS DATA
# -------------------------
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)


Found 715 images belonging to 1 classes.
Found 21 images belonging to 1 classes.


In [6]:
# -------------------------
# 4. LOAD PRETRAINED VGG16 MODEL (without top layers)
# -------------------------
base_model = VGG16(
    weights=None,  # we’ll load custom weights manually
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.load_weights(weights_path)


In [7]:
# -------------------------
# 5. FREEZE LOWER LAYERS
# -------------------------
for layer in base_model.layers:
    layer.trainable = False

# -------------------------
# 6. ADD CUSTOM CLASSIFIER
# -------------------------
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

In [8]:
# -------------------------
# 7. COMPILE MODEL
# -------------------------
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# -------------------------
# 8. TRAIN CLASSIFIER LAYERS
# -------------------------
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


Epoch 1/5


C:\Users\saira\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\ops\nn.py:946: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\saira\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 42s 910ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 40s 879ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 862ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 862ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 862ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [8]:
# -------------------------
# 9. FINE-TUNE (OPTIONAL)
# -------------------------
# Unfreeze last few convolutional layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=1
)

# -------------------------
# 10. EVALUATE MODEL
# -------------------------
loss, acc = model.evaluate(test_data)
print(f"\n✅ Test Accuracy: {acc:.2f}")

Epoch 1/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.9894 - val_loss: 0.0380
Epoch 2/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9894 - val_loss: 0.0214
Epoch 3/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 1.0000 - loss: 3.2536e-04 - val_accuracy: 0.9894 - val_loss: 0.0316
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 979ms/step - accuracy: 0.9929 - loss: 0.0223

✅ Test Accuracy: 0.99


In [10]:
# -------------------------
# 11. SAVE MODEL WEIGHTS
# -------------------------
model.save_weights("fine_tuned_vgg16.weights.h5")
print("\nModel weights saved successfully!")

# -------------------------
# 12. MAKE SAMPLE PREDICTIONS
# -------------------------
import numpy as np
sample_batch, labels = next(test_data)
predictions = np.argmax(model.predict(sample_batch), axis=1)
print("\nPredicted Class IDs:", predictions)



Model weights saved successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 912ms/step

Predicted Class IDs: [0 0 0 0 2 0 0 0 1 0 0 0 2 0 0 0]
